In [1]:
#import sys   ## (From Tom's tutorial on stashing api keys)
#sys.path

In [2]:
import config_from_pydata_env

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# from api_keys import g_key   ## (Substituted with below)
from config_from_pydata_env import g_key  # From Tom's tutorial on stashing keys, 10/13 video 1

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Load the csv exported in Part I to a DataFrame:
weather_file = "../Output/weather_df.csv"
vacation_df = pd.read_csv(weather_file)
vacation_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,caceres,39.52,-6.37,68.00,33,5,7.00,ES
1,mataura,-46.19,168.86,35.01,92,77,3.00,NZ
2,batagay,67.63,134.63,15.66,95,100,3.98,RU
3,albany,42.60,-73.97,64.99,64,0,6.73,US
4,monopoli,40.96,17.29,66.00,88,75,10.29,IT


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# (Day 3 Activities 6, 10) 

# Store gkey:
gmaps.configure(api_key = g_key)

# Store locations, lat, lng:
locations = vacation_df[["Latitude", "Longitude"]].astype(float)

# Store humidity as weights:
weights = vacation_df["Humidity"].astype(float)

In [7]:
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

# Size and create figure:
figure_layout = {
    "width": "800px",
    "height": "600px",
    "border": "1px solid black",
    "padding": "1px"
}
figure = gmaps.figure(layout = figure_layout, center = (0, 0), zoom_level = 2)

# Add heatmap layer:
heat_layer = gmaps.heatmap_layer(locations, weights, max_intensity=50)
figure.add_layer(heat_layer)

# Show:
figure

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Create df with ideal conditions:
# Moderate temps, some nice clouds, and not too humid:

ideal_cities = vacation_df[vacation_df["Max Temp"]     < 80]
ideal_cities = ideal_cities[ideal_cities["Max Temp"]   > 65]
ideal_cities = ideal_cities[ideal_cities["Wind Speed"] < 15]
ideal_cities = ideal_cities[ideal_cities["Cloudiness"] >=20]
ideal_cities = ideal_cities[ideal_cities["Cloudiness"] < 40]
ideal_cities = ideal_cities[ideal_cities["Humidity"]   < 75]

ideal_cities
# 9 cities

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
53,cape town,-33.93,18.42,72.00,64,33,9.17,ZA
56,chuy,-33.70,-53.46,68.18,64,32,7.38,UY
57,prykolotne,50.16,37.35,67.28,49,28,4.99,UA
217,port pirie,-33.18,138.02,73.40,31,26,11.41,AU
280,pisco,-13.70,-76.22,66.20,72,20,10.29,PE
307,coahuayana,18.73,-103.68,77.00,73,20,11.41,MX
391,bumba,2.18,22.47,78.78,69,20,0.34,CD
488,talara,-4.58,-81.27,75.20,69,26,12.75,PE
511,filadelfia,39.95,-75.16,69.01,48,20,10.29,US


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# (Used Day 3, Activity 5: Airport Ratings)

# Store info into a variable named hotel_df:
hotel_df = pd.DataFrame(ideal_cities)
hotel_df["Hotel Name"] = ""
#hotel_df.head()

# Set parameters to search for hotels with 5000 meters:
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

# change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# make request and print url
    name_address = requests.get(base_url, params=params)

# convert to json:
    name_address = name_address.json()
# print(json.dumps(name_address, indent=4, sort_keys=True))

# Store the first Hotel result into the DataFrame.    
# Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "None Found"
        print("No hotel found in {row['City']}.")

# Renaming columns.  The Cell below says not to change anything, but uses "Lat" and "Lng".
# I had been using "Latitude" and "Longitude" the whole time.
hotel_df = hotel_df.rename(columns = {"Latitude": "Lat", "Longitude": "Lng"})

# Show Results:
hotel_df

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
#hotel_info

In [ ]:
# Add marker layer ontop of heat map
# Day 3, Activity 9

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(markers)

# Display figure
figure